# Week 7

This is the set of programming assignments for week 7.
    
The tasks cover the following topics:
- quantum phase estimation

We recommend to solve the PhaseEstimation kata from https://github.com/Microsoft/QuantumKatas before doing these assignments.

## Part I. Quantum phase estimation (QPE)

### Task 1.1. Eigenstates of the Hadamard gate.
**Inputs:**
1. a qubit in $|0\rangle$ state.
2. an integer `state` indicating which eigenstate to prepare (0 or 1).

**Goal:** prepare one of the eigenstates of the Hadamard gate ($|\psi_0\rangle$ or $|\psi_1\rangle$). The eigenstate $|\psi_0\rangle$ (prepared for `state = 0`) should be the one with eigenvalue -1, $|\psi_1\rangle$ (prepared for `state = 1`) - the one with eigenvalue 1.

In [16]:
%kata T11_Test

open Microsoft.Quantum.Math;

operation Task11 (q : Qubit, state : Int) : Unit is Adj {
    // Note that your implementation has to be adjointable.
    // ...
    let num = (state == 0) ? (ArcTan(1.0/(1.0 + Sqrt(2.0)*-1.0))) | (ArcTan(1.0/(1.0 + Sqrt(2.0)*1.0)));
    Ry(2.0 * num, q);
}

Success!

### Task 1.2. Speed up phase estimation oracle for T gates
**Input:**  A positive integer $P$.

**Output:** A single-qubit unitary equal to $T$ gate raised to the power of $P$.

You can use at most one $T$ gate for any value of $P$.
You can not use arbitrary rotation gates or allocate extra qubits.

In [20]:
operation PowerUnitary (P : Int, q: Qubit) : Unit is Adj+Ctl {
    
    if (P % 2  == 1) {
        T(q);
    }
    if ((P >>> 1) % 2 == 1){ // a >>> b is equivalent to RightShiftedI(a, b);
        S(q);
    }
    if ((P >>> 2) % 2 == 1){
        Z(q);
    }
}

PowerUnitary

In [21]:
%kata T12_Test

function Task12 (P : Int) : (Qubit => Unit is Adj+Ctl) {
    // Currently this function returns the identity gate for the sake of being able to compile the code.
    // You will need to return your own unitary instead of I.
    
    return PowerUnitary(P, _);
}

Testing power 0...
Testing power 1...
Testing power 2...
Testing power 3...
Testing power 4...
Testing power 5...
Testing power 6...
Testing power 7...
Testing power 8...
Testing power 9...
Testing power 10...
Testing power 11...
Testing power 12...
Testing power 13...
Testing power 14...
Testing power 15...
Testing power 16...
Testing power 17...
Testing power 18...
Testing power 19...
Testing power 20...
Testing power 21...
Testing power 22...
Testing power 23...
Testing power 24...
Testing power 25...
Testing power 26...
Testing power 27...
Testing power 28...
Testing power 29...
Testing power 30...
Testing power 31...
Testing power 32...


Success!

### Task 1.3. Quantum version of unitary power
**Inputs:**
1. a single-qubit unitary $U$.
2. N qubits in an arbitrary state $|k\rangle$.
3. a qubit in an arbitrary state $|\psi\rangle$ (not necessarily an eigenstate of $U$).

**Goal:** apply the k-th power of unitary $U$ to the target qubit, i.e., transform the state $|k\rangle|\psi\rangle$ into the state $|k\rangle U^k |\psi\rangle$.  
Here the integer k is stored in little endian format (least significant bit first).

Note that this task is similar to the oracle tasks:
`powerRegister` can be in superposition, so you should not measure it to obtain the value of $k$.

> For example, for $U = S$ the state $(|00\rangle + |01\rangle + |10\rangle + |11\rangle) \otimes |1\rangle$ should be transformed into $|00\rangle \otimes |1\rangle + |10\rangle \otimes S|1\rangle + |01\rangle \otimes S^2 |1\rangle + |11\rangle \otimes S^3 |1\rangle = (|00\rangle + i |10\rangle - |01\rangle - i|11\rangle) \otimes |1\rangle$.

In [25]:
%kata T13_Test

open Microsoft.Quantum.Canon;

operation Task13 (U : (Qubit => Unit is Adj+Ctl), 
                  powerRegister : Qubit[], 
                  target : Qubit) : Unit is Adj {
    // Note that your implementation has to be adjointable.
    // Hint: this circuit is part of the quantum phase estimation circuit presented in the lecture
    
    for i in 0 .. Length(powerRegister) - 1 {
        Controlled (OperationPowCA(U, 1 <<< i))([powerRegister[i]], target);
    } 
}

Success!

### Task 1.4. Reverse-engineer QPE
**Input:**  A floating-point number $\phi$ in `[0, 1)` interval.

**Output:** A tuple of two single-qubit unitaries `(U, P)` which, when passed to the QPE implementation from task 1.4 in PhaseEstimation kata, produce $\phi$.  
In other words, find a unitary and its eigenstate which has the eigenvalue $exp(2i\pi \phi)$.

> For example, 
* if `phase = 0.0`, you can return `(Z, I)` to represent eigenstate $|0\rangle$ with eigenvalue $1 = exp(2πi · 0.0)$; 
* if `phase = 0.5`, you can return `(Z, X)` to represent eigenstate $|1\rangle$ with eigenvalue $-1 = exp(2\pi i · 0.5)$.

QPE will be invoked with $n = 8$ bits of precision. The returned value has to be accurate within the absolute error of $0.01$.

The test will be executed once for each value of $\phi$.

In [50]:
%kata T14_Test

open Microsoft.Quantum.Convert;
open Microsoft.Quantum.Math;

function Task14 (φ : Double) : ((Qubit => Unit is Adj+Ctl), (Qubit => Unit is Adj)) {
    // Hint: Do not try to create a pair that will give φ exactly. Instead, consider the following questions:
    //       When does QPE procedure fail? What can you do to make sure it does not fail on the unitaries you return?
    // ...
    
    let phase = IntAsDouble(Round(φ * 256.0))/256.0;
    let theta = 2.0 * 3.1415926535 * phase; // Dunno why PI doesn't work...
    
    return (R1(theta, _), X);
}

Success!